# GPTでレビュー分類

In [81]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [82]:
# データのパス
data_path = '/Users/manqueenmannequin/TW_otameshi/project_root/data/raw/android.json'

# データ読み込み
with open(data_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    
df = pd.DataFrame(data['data'])

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

データサイズ: (1000, 14)


,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,None,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[]
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,None,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,None,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,None,None,None,0,[]
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,None,CMが多い ５秒から30秒動画が流れます,None,None,9.6.2,0,[]
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,None,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[]


In [83]:
import openai
import os

# OpenAI APIのキーを環境変数から取得
openai.api_key = os.getenv("OPENAI_API_KEY")

#print(os.getenv("OPENAI_API_KEY"))  # APIキーが表示されれば成功

In [84]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

# テスト用の文章を生成
try:
    chat_completion = client.chat.completions.create(
        model="gpt-4o",  # 正しいモデル名
        messages=[
            {
                "role": "system",
                "content": "レビューの重要な情報を保持しながら、ポジティブな意見とネガティブな意見に分類しクリーニングしてください。",
            },
            {
                "role": "user",
                "content": "レビュー: このアプリ、まあまあだけどy使いにくい😄部分もあります。",
            }
        ]
    )
    # 応答内容を表示
    print(chat_completion.choices[0].message.content)
except openai.error.OpenAIError as e:
    print(f"OpenAI API エラー: {e}")

ポジティブな意見:
- まあまあ

ネガティブな意見:
- 使いにくい部分がある


In [85]:
# GPTを用いたクリーニング用の関数を定義

def clean_review_with_gpt(review):
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4o",  # 正しいモデル名
            messages=[
            {
                "role": "system",
                "content": "レビューの重要な情報を出来るだけ保持しながら、ポジティブな意見とネガティブな意見に分類してください。その際，必要であればクリーニングしてください。特にポジティブな意見またはネガティブな意見がない場合は「特になし」と答えてください。",
            },
            {
                "role": "user",
                "content": f"レビュー: {review}",
            }
            ]
        )
        return chat_completion.choices[0].message.content
    except openai.error.OpenAIError as e:
        print(f"OpenAI API エラー: {e}")
        return None
    
# テスト用のレビュー
review = "このアプリ、まあまあだけど使いにくい😄部分もあります。"

# レビューのクリーニング
cleaned_review = clean_review_with_gpt(review)

# クリーニング後のレビューを表示
print(cleaned_review)

ポジティブな意見: 特になし

ネガティブな意見: 使いにくい部分がある


In [86]:
# dfのtextに対して，clean_review_with_gpt関数を適用．ポジティブな意見とネガティブな意見を分類しクリーニング，結果をポジティブな意見とネガティブな意見の二つの新たなカラムとして追加
def extract_reviews(cleaned_review):
    positive_review = re.search(r'ポジティブな意見:\s*(.*)', cleaned_review)
    negative_review = re.search(r'ネガティブな意見:\s*(.*)', cleaned_review)
    return positive_review.group(1) if positive_review else '', negative_review.group(1) if negative_review else ''

# まずは最初の10件に対してクリーニングを行う
#df[['positive_review', 'negative_review']] = df['text'].head(10).apply(lambda x: pd.Series(extract_reviews(clean_review_with_gpt(x))))

df[['positive_review', 'negative_review']] = df['text'].apply(lambda x: pd.Series(extract_reviews(clean_review_with_gpt(x))))

# データの確認
df

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,None,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],特になし,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,None,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,特になし,- 指摘した問題が改善されていない
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,None,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,None,None,None,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,特になし。
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,None,CMが多い ５秒から30秒動画が流れます,None,None,9.6.2,0,[],特になし,CMが多く、5秒から30秒の動画が流れる
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,None,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,特になし。
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,a9462ebd-6ac7-4920-8285-5922664d0592,印南勤,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-04-10T15:56:08.523Z,4,4,https://play.google.com/store/apps/details?id=...,None,つながりがかなり良い,2024-04-14T06:40:17.733Z,いつもアプリをお使いいただき、ありがとうございます！\n今後とも、タウンWiFiをよろしくお...,8.31.1,0,"[{'criteria': 'vaf_phase1_file_sharing', 'rati...",つながりがかなり良い,特になし
996,0733b493-a295-4cee-931c-144fcc1e44a0,三河のどか,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-04-10T08:51:49.680Z,1,1,https://play.google.com/store/apps/details?id=...,None,ビックリするほどバッテリーの消耗が激しい。使わない時もバックグラウンドで動いているようで使わ...,None,None,None,8,[],特になし,- バッテリーの消耗が激しい
997,90f66974-f8f9-44ef-9079-707cfd207c82,村上稔,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-04-10T08:24:58.380Z,3,3,https://play.google.com/store/apps/details?id=...,None,又近い内に,None,None,None,0,[],特になし,特になし
998,c417d407-b5f8-40c0-b074-f331e8aab9a6,沼津のトシ,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-04-09T23:26:14.886Z,3,3,https://play.google.com/store/apps/details?id=...,None,スタンプをせっせと貯めていますが、いかんせん交換率が凄く低くて、3万ポイントがアマゾンポイン...,None,None,8.31.1,3,[],特になし,スタンプの交換率が低く、3万ポイントをアマゾンポイントに変えると300ポイントで、13万ポイ...


In [87]:
# クリーニング後のデータを保存
df.to_csv('/Users/manqueenmannequin/TW_otameshi/project_root/data/processed/android_cleaned.csv', index=False)